# PySpark Linear Regression

Based on **Krish Naik** video: https://www.youtube.com/watch?v=_C8kWso4ne4

## DataBricks version (no internet notes)

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Practise').getOrCreate()

23/06/25 20:07:29 WARN Utils: Your hostname, Wiktors-MacBook-Air.local resolves to a loopback address: 127.0.0.1, but we couldn't find any external IP address!
23/06/25 20:07:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/25 20:07:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# File location and type
file_location = "data/tips.csv" # Databricks file_location "/FileStore/tables//test1.csv"
file_type = "csv"

# Theapplied options are for CSV files. For other file types, these will be ignored.
df = spark.read.csv(file_location, header=True, inferSchema=True)

In [6]:
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [7]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [8]:
# Handling Categorical Features

from pyspark.ml.feature import StringIndexer

In [14]:
indexer = StringIndexer(inputCols=["sex", "smoker", "day", "time"], outputCols=["sex_indexed", "smoker_indexed", "day_indexed", "time_indexed"])
df_r = indexer.fit(df).transform(df)

In [19]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=[ 
                'tip',
                'size',
                'sex_indexed',
                'smoker_indexed',
                'day_indexed',
                'time_indexed'],
                outputCol = "Independent Features")

output = featureassembler.transform(df_r)

In [23]:
output.select('Independent Features').show()

+--------------------+
|Independent Features|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [24]:
finalized_data = output.select("Independent Features", "total_bill")

In [25]:
finalized_data.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [27]:
from pyspark.ml.regression import LinearRegression
# train test split
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol = "Independent Features", labelCol="total_bill")
regressor = regressor.fit(train_data)

23/06/25 21:30:14 WARN Instrumentation: [91aaca5e] regParam is zero, which might cause numerical instability and overfitting.
23/06/25 21:30:14 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/06/25 21:30:14 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [28]:
regressor.coefficients

DenseVector([3.3, 3.7418, -0.6045, 2.46, -0.1957, -0.9748])

In [31]:
regressor.intercept

-0.04009673452175394

In [32]:
# Predictions
pred_results = regressor.evaluate(test_data)

In [34]:
# Final comparison
pred_results.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.75,2.0])|     17.82|13.218519205936602|
| (6,[0,1],[2.0,2.0])|     12.69|14.043519575363714|
|(6,[0,1],[2.31,3.0])|     18.69| 18.80832671068762|
| (6,[0,1],[2.5,4.0])|     18.35|23.177133668686515|
|(6,[0,1],[2.64,3.0])|     17.59|19.897327198331407|
| (6,[0,1],[3.0,2.0])|      14.0| 17.34352105307216|
|(6,[0,1],[3.27,2.0])|     17.78| 18.23452145205344|
| (6,[0,1],[3.6,3.0])|     24.06|23.065328616931517|
|[1.0,1.0,1.0,0.0,...|      7.25| 6.397186325748818|
|[1.17,2.0,0.0,1.0...|     32.83|13.764560934337165|
|[1.25,2.0,1.0,0.0...|      8.51| 9.597733133070365|
|[1.36,3.0,1.0,0.0...|     18.64|13.702539972852582|
|[1.5,2.0,0.0,1.0,...|     12.03|14.266335351949493|
|[1.63,2.0,1.0,0.0...|     11.87|10.851733694599572|
|[1.66,3.0,0.0,0.0...|     10.34| 16.46758372683331|
|[1.98,2.0,0.0,1.0...|     11.02| 16.437562131

In [37]:
### Performance Metrics
pred_results.r2, pred_results.meanAbsoluteError, pred_results.meanSquaredError

(0.36200040519452903, 4.7770765433873414, 42.141770151474404)

### TO DO

How I ca  save this particular file in pickle format or in a temporary model pickle file. .save but check in documentation